# ResNET
#### Experiments using Resnet models and fine-tuning with image dataset.

In [1]:
from scripts.models.resnet import ResNetClassifier

In [2]:
import os

from pathlib import Path

from dotenv import load_dotenv
load_dotenv()
root_data = os.getenv("KAGGLE_FILES_DIR")
dataset_path = Path(os.getcwd(), "..", root_data, 'processed')

#### Prepare trainer, callbacks from Pytorch Lightning

In [3]:
import pytorch_lightning as pl

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="../models/resnet18",
    filename="resnet18-model-{epoch}-{val_loss:.3f}-{val_acc:0.3f}",
    monitor="val_loss",
    save_top_k=2,
    mode="min",
    save_last=True,
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=True,
    mode='min'
)

# Instantiate Lightning trainer and train model
trainer_args = {
    "accelerator": "mps",
    "max_epochs": 100,
    "callbacks": [checkpoint_callback, early_stopping],
    "precision": 32,
}
trainer = pl.Trainer(**trainer_args)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### ResNet18 Model
- binary classification
- optimizer: Adam
- learning rate: 1e-4
- tune all layers
- loss function: BCEWithLogitsLoss

In [6]:
resnet18 = ResNetClassifier(
    num_classes=1,
    resnet_version=18,
    train_path=Path(dataset_path, "train"),
    val_path=Path(dataset_path, "val"),
    test_path=Path(dataset_path, "test"),
    optimizer="adam",
    lr=1e-4,
    batch_size=32,
    tune_fc_only=False,
)


In [7]:
trainer.fit(resnet18)


  | Name         | Type              | Params
---------------------------------------------------
0 | loss_fn      | BCEWithLogitsLoss | 0     
1 | accuracy     | BinaryAccuracy    | 0     
2 | resnet_model | ResNet            | 11.2 M
---------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.708    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.560


...

Metric val_loss improved by 0.001 >= min_delta = 0.001. New best score: 0.461


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.461. Signaling Trainer to stop.


In [8]:
trainer.test(resnet18)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7771162390708923     │
│         test_loss         │    0.6620050668716431     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.6620050668716431, 'test_acc': 0.7771162390708923}]

Results are promising, but we can try to improve the model by fine-tuning the last layer only.

In [12]:

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="../models/resnet18-fc-only",
    filename="resnet18-model-{epoch}-{val_loss:.3f}-{val_acc:0.3f}",
    monitor="val_loss",
    save_top_k=2,
    mode="min",
    save_last=True,
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=True,
    mode='min'
)

# Instantiate Lightning trainer and train model
trainer_args = {
    "accelerator": "mps",
    "max_epochs": 100,
    "callbacks": [checkpoint_callback, early_stopping],
    "precision": 32,
}
trainer = pl.Trainer(**trainer_args)


resnet18 = ResNetClassifier(
    num_classes=1,
    resnet_version=18,
    train_path=Path(dataset_path, "train"),
    val_path=Path(dataset_path, "val"),
    test_path=Path(dataset_path, "test"),
    optimizer="adam",
    lr=1e-4,
    batch_size=32,
    tune_fc_only=True,
)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(resnet18)


  | Name         | Type              | Params
---------------------------------------------------
0 | loss_fn      | BCEWithLogitsLoss | 0     
1 | accuracy     | BinaryAccuracy    | 0     
2 | resnet_model | ResNet            | 11.2 M
---------------------------------------------------
513       Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.708    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.582


...

Metric val_loss improved by 0.001 >= min_delta = 0.001. New best score: 0.576


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.576. Signaling Trainer to stop.


In [14]:
trainer.test(resnet18)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7273219227790833     │
│         test_loss         │    0.5764658451080322     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5764658451080322, 'test_acc': 0.7273219227790833}]

Results are worse, so return with training all layers.
Next, I check SGD optimizer.


In [20]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="../models/resnet18-sgd",
    filename="resnet18-model-{epoch}-{val_loss:.3f}-{val_acc:0.3f}",
    monitor="val_loss",
    save_top_k=2,
    mode="min",
    save_last=True,
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=True,
    mode='min'
)

trainer_args = {
    "accelerator": "mps",
    "max_epochs": 100,
    "callbacks": [checkpoint_callback, early_stopping],
    "precision": 32,
}
trainer = pl.Trainer(**trainer_args)


resnet18 = ResNetClassifier(
    num_classes=1,
    resnet_version=18,
    train_path=Path(dataset_path, "train"),
    val_path=Path(dataset_path, "val"),
    test_path=Path(dataset_path, "test"),
    optimizer="sgd",
    lr=1e-4,
    batch_size=32,
    tune_fc_only=False,
)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
trainer.fit(resnet18)


  | Name         | Type              | Params
---------------------------------------------------
0 | loss_fn      | BCEWithLogitsLoss | 0     
1 | accuracy     | BinaryAccuracy    | 0     
2 | resnet_model | ResNet            | 11.2 M
---------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.708    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.582


...

Metric val_loss improved by 0.002 >= min_delta = 0.001. New best score: 0.561


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.561. Signaling Trainer to stop.


In [18]:
trainer.test(resnet18)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7389045357704163     │
│         test_loss         │    0.5607972145080566     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5607972145080566, 'test_acc': 0.7389045357704163}]

Results are worse and training time is much longer, so now with AdamW optimizer.

In [5]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="../models/resnet18-adamw",
    filename="resnet18-model-{epoch}-{val_loss:.3f}-{val_acc:0.3f}",
    monitor="val_loss",
    save_top_k=2,
    mode="min",
    save_last=True,
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=True,
    mode='min'
)

trainer_args = {
    "accelerator": "mps",
    "max_epochs": 100,
    "callbacks": [checkpoint_callback, early_stopping],
    "precision": 32,
}
trainer = pl.Trainer(**trainer_args)


resnet18 = ResNetClassifier(
    num_classes=1,
    resnet_version=18,
    train_path=Path(dataset_path, "train"),
    val_path=Path(dataset_path, "val"),
    test_path=Path(dataset_path, "test"),
    optimizer="adamw",
    lr=1e-4,
    batch_size=32,
    tune_fc_only=False,
)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(resnet18)


  | Name         | Type              | Params
---------------------------------------------------
0 | loss_fn      | BCEWithLogitsLoss | 0     
1 | accuracy     | BinaryAccuracy    | 0     
2 | resnet_model | ResNet            | 11.2 M
---------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.708    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.570


...

Metric val_loss improved by 0.011 >= min_delta = 0.001. New best score: 0.462


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.462. Signaling Trainer to stop.


In [7]:
trainer.test(resnet18)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7824204564094543     │
│         test_loss         │    0.5112970471382141     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5112970471382141, 'test_acc': 0.7824204564094543}]

Accuracy 0.78 - a bit better than Adam optimizer. So AdamW stays.  
Next I will try learning rate od 1e-3.

In [8]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="../models/resnet18-adamw-1e-3",
    filename="resnet18-model-{epoch}-{val_loss:.3f}-{val_acc:0.3f}",
    monitor="val_loss",
    save_top_k=2,
    mode="min",
    save_last=True,
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=True,
    mode='min'
)

trainer_args = {
    "accelerator": "mps",
    "max_epochs": 100,
    "callbacks": [checkpoint_callback, early_stopping],
    "precision": 32,
}
trainer = pl.Trainer(**trainer_args)


resnet18 = ResNetClassifier(
    num_classes=1,
    resnet_version=18,
    train_path=Path(dataset_path, "train"),
    val_path=Path(dataset_path, "val"),
    test_path=Path(dataset_path, "test"),
    optimizer="adamw",
    lr=1e-3,
    batch_size=32,
    tune_fc_only=False,
)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(resnet18)


  | Name         | Type              | Params
---------------------------------------------------
0 | loss_fn      | BCEWithLogitsLoss | 0     
1 | accuracy     | BinaryAccuracy    | 0     
2 | resnet_model | ResNet            | 11.2 M
---------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.708    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.591


...

Metric val_loss improved by 0.005 >= min_delta = 0.001. New best score: 0.461


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.461. Signaling Trainer to stop.


In [12]:
trainer.test(resnet18)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7792812585830688     │
│         test_loss         │    0.4995814263820648     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.4995814263820648, 'test_acc': 0.7792812585830688}]

Previous model has the same results but with much shorter training time.
So I will stick with 1e-4 learning rate, AdamW optimizer and training all layers.
Next is changing ResNet size.

In [14]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="../models/resnet34-adamw",
    filename="resnet34-model-{epoch}-{val_loss:.3f}-{val_acc:0.3f}",
    monitor="val_loss",
    save_top_k=2,
    mode="min",
    save_last=True,
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=True,
    mode='min'
)

trainer_args = {
    "accelerator": "mps",
    "max_epochs": 100,
    "callbacks": [checkpoint_callback, early_stopping],
    "precision": 32,
}
trainer = pl.Trainer(**trainer_args)


resnet34 = ResNetClassifier(
    num_classes=1,
    resnet_version=34,
    train_path=Path(dataset_path, "train"),
    val_path=Path(dataset_path, "val"),
    test_path=Path(dataset_path, "test"),
    optimizer="adamw",
    lr=1e-4,
    batch_size=32,
    tune_fc_only=False,
)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(resnet34)


  | Name         | Type              | Params
---------------------------------------------------
0 | loss_fn      | BCEWithLogitsLoss | 0     
1 | accuracy     | BinaryAccuracy    | 0     
2 | resnet_model | ResNet            | 21.3 M
---------------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.141    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.558


...

Metric val_loss improved by 0.005 >= min_delta = 0.001. New best score: 0.454


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.454. Signaling Trainer to stop.


In [16]:
trainer.test(resnet34)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7960597276687622     │
│         test_loss         │    0.47373509407043457    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.47373509407043457, 'test_acc': 0.7960597276687622}]

Result are better, but not much.
I will try ResNet50 model and decide on the best model.

In [6]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="../models/resnet50-adamw",
    filename="resnet50-model-{epoch}-{val_loss:.3f}-{val_acc:0.3f}",
    monitor="val_loss",
    save_top_k=2,
    mode="min",
    save_last=True,
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=True,
    mode='min'
)

trainer_args = {
    "accelerator": "mps",
    "max_epochs": 100,
    "callbacks": [checkpoint_callback, early_stopping],
    "precision": 32,
}
trainer = pl.Trainer(**trainer_args)


resnet50 = ResNetClassifier(
    num_classes=1,
    resnet_version=50,
    train_path=Path(dataset_path, "train"),
    val_path=Path(dataset_path, "val"),
    test_path=Path(dataset_path, "test"),
    optimizer="adamw",
    lr=1e-4,
    batch_size=32,
    tune_fc_only=False,
)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(resnet50)


  | Name         | Type              | Params
---------------------------------------------------
0 | loss_fn      | BCEWithLogitsLoss | 0     
1 | accuracy     | BinaryAccuracy    | 0     
2 | resnet_model | ResNet            | 23.5 M
---------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.040    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.564


...

Metric val_loss improved by 0.003 >= min_delta = 0.001. New best score: 0.464


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.464. Signaling Trainer to stop.


In [8]:
trainer.test(resnet50)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7820956707000732     │
│         test_loss         │    0.5501084923744202     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5501084923744202, 'test_acc': 0.7820956707000732}]

Resnet50 model gave similar result as Resnet34, but training time was much longer.  
So my final model will be Resnet34 with AdamW optimizer.

In [4]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="../models/resnet34-adamw",
    filename="resnet34-model-{epoch}-{val_loss:.3f}-{val_acc:0.3f}",
    monitor="val_loss",
    save_top_k=2,
    mode="min",
    save_last=True,
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=True,
    mode='min'
)

trainer_args = {
    "accelerator": "mps",
    "max_epochs": 100,
    "callbacks": [checkpoint_callback, early_stopping],
    "precision": 32,
}
trainer = pl.Trainer(**trainer_args)

best_model = ResNetClassifier.load_from_checkpoint(
    "../models/resnet34-adamw/resnet34-model-epoch=9-val_loss=0.454-val_acc=0.810.ckpt"
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
trainer.test(best_model)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8039618730545044     │
│         test_loss         │    0.4579312205314636     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.4579312205314636, 'test_acc': 0.8039618730545044}]